In [1]:
#dependencies 
import pandas as pd
import numpy as np
import re
from thefuzz import fuzz
from thefuzz import process as fwp
from sklearn.cluster import KMeans

C:\Users\TDC\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


#### 1- Load and Prepare the data

##### 1.  Load the data:

In [2]:
execl_filename="./data.xlsx"

data_sheet="Sheet1"
models_sheet="Sheet2"
#load excel
excel_file = pd.ExcelFile(execl_filename)
#parse sheet 1
data_df = excel_file.parse(data_sheet,converters={"title":str,"model":str,"mark":str,"transmition_type":str})
models_df= excel_file.parse(models_sheet,converters={"model":str,"mark":str})


#####    2. Prepare and clean models data

        * Clean and drop duplicates

In [3]:
#sort and clean dublicates 
models_df.drop_duplicates(subset=["mark","model","year_model",'model_comp',"version_comp",'price'],keep="first",inplace=True)
models_df.sort_values( ["mark","model","year_model","price"],inplace=True)

        * Add transmition type label to the models data:

In [4]:
def parse_model_transmission_type(row):
    version=row.version_comp.lower()
    ver=version.lower()
    
    autop=r'\bat\d{0,1}p{0,1}\b|\b\d{0,1}at\b|\be{1}at\d{1}\b'
    manp=r'\bmt\d{0,1}p{0,1}\b|\b\d{0,1}mt\b|\bhpmt\b'
    
    if re.search(autop, ver):
        return "auto"
    

    elif re.search(manp, ver): 
        return "manual"
    # print("not found",version)
    return np.nan
 

In [5]:
#add a transmition type lable (column)
models_df["transmission_type"]=models_df.apply(parse_model_transmission_type, axis='columns')
models_df.head(5)

,model_ id,year_model,mark,model,model_comp,version_comp,tipo_carroceria,price,banda_min,banda_max,km_referencia,publicaciones,rank1,rank2,transmission_type
0,15,1990,ACURA,INTEGRA,INTEGRA,1.6 LS MT 4P,SEDAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manual
1,16,1990,ACURA,INTEGRA,INTEGRA,1.8 LS MT 4P,SEDAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manual
2,17,1990,ACURA,INTEGRA,INTEGRA,1.6 MT 4P,SEDAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manual
3,18,1990,ACURA,INTEGRA,INTEGRA,1.8 MT 4P,SEDAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manual
4,19,1990,ACURA,INTEGRA,INTEGRA,1.8 RS MT 4P,SEDAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manual


* Add the ranks (rank1 and rank2) columns to the models data:

>Rank1: group the data in (mark,model,year) groups and rank each item in that group based on it's price

In [6]:
#add  rank1
models_df["rank1"]=models_df.groupby(["mark","model","year_model"])["price"].rank("dense")

>Rank2: apply KMeans clustering to each group to cluster items in that group to 3 diffrent clusters (ranks)

In [7]:
#the ranking function
def rank2(g):
    #get the maximun rank1 for this groupby
    #the max represents how many diffrent price trag there are in this group 
    max_r1=g.rank1.max()
    
    if pd.isnull(max_r1):
        #max is null means there are no prices on this group (models does not have  price info)
        g["rank2"]=max_r1
    elif  max_r1 <= 3:
        # if there are 3 or less diffrent prices , simply set the rank2 to the same values as rank1
        g["rank2"]=g["rank1"]
    else :
        # if there are more than 3 diffrent prices, apply a kmeans clustering
        # to group the prices into 3 ranks
        g2=g.loc[g["rank1"].notna()]   
        km=KMeans(n_clusters=3)
        prices=g2["price"].values.reshape(-1, 1) 
        rank2=km.fit_predict(prices)
        g2["rank2"]=rank2 
        # this makes sure the clusters are lables correctly where the group1(rank1) is the one
        # with the lowest prices and the group3 with the highest
        g2['max'] = g2.groupby('rank2')['price'].transform('max')
        g2["rank2"]=g2["max"].rank(method="dense")
        g=g.merge(g2,how ="left",left_index=True,right_index=True)

        

    return g["rank2"].to_frame()

In [8]:
#apply the ranking function the dataframe 
models_df["rank2"]=models_df.groupby(["mark","model","year_model"])[["price","rank1"]].apply(rank2)

* Index the models data the models data by "mark","model","year_model" for esier access:

In [9]:
index= ["mark","model","year_model"]
models_df.set_index(index, inplace=True)
models_df.sort_index(inplace=True)

In [10]:
#check if everything worked as intendes
models_df.loc['MAZDA','3',2019]

model_ id model_comp  \
mark  model year_model                         
MAZDA 3     2019            65586      NEW 3   
            2019            65587      NEW 3   
            2019            65588      NEW 3   
            2019            65590      NEW 3   
            2019            65589      NEW 3   
            2019            65591      NEW 3   
            2019            70140      NEW 3   
            2019            70139      NEW 3   
            2019            70138      NEW 3   
            2019            65593      NEW 3   
            2019            70137      NEW 3   
            2019            70135      NEW 3   
            2019            70136      NEW 3   
            2019            70134      NEW 3   
            2019            65592      NEW 3   
            2019            65594      NEW 3   

                                                       version_comp  \
mark  model year_model                                                
MAZDA 3     2019                                 1.6 S SEDAN 5MT 4P   
            2019                                 1.6 S SEDAN 4AT 4P   
            2019                                 2.0 V SEDAN 6MT 4P   
            2019                              2.0 V SEDAN SR 6AT 4P   
            2019                              2.0 V SPORT HB 6MT 5P   
            2019                           2.0 V SPORT HB SR 6AT 5P   
            2019                                 2.0 V SPORT 6AT 5P   
            2019                          2.5 GT SEDAN SR GPS AT 4P   
            2019               2.5 GT SEDAN SR AUDIO BOSE GPS AT 4P   
            2019        2.5 GT SEDAN SR AUDIO BOSE CUERO GPS 6AT 4P   
            2019                             2.5 GT SPORT SR 6AT 5P   
            2019                    2.5 GT SPORT AUDIO BOSE 6EAT 5P   
            2019                     2.5 GT SPORT AUDIO BOSE 6MT 5P   
            2019                     2.5 GT SPORT AUDIO BOSE 6AT 5P   
            2019                     2.5 GT SPORT HB SUNROOF 6MT 5P   
            2019                     2.5 GT SPORT HB SUNROOF 6AT 5P   

                       tipo_carroceria       price   banda_min   banda_max  \
mark  model year_model                                                       
MAZDA 3     2019                 SEDAN  10800000.0  10420000.0  11190000.0   
            2019                 SEDAN  10980000.0  10600000.0  11360000.0   
            2019                 SEDAN  13350000.0  12870000.0  13820000.0   
            2019                 SEDAN  13640000.0  13170000.0  14120000.0   
            2019             HATCHBACK  14060000.0  13440000.0  14680000.0   
            2019             HATCHBACK  14430000.0  13810000.0  15050000.0   
            2019             HATCHBACK  14440000.0  13820000.0  15060000.0   
            2019                 SEDAN  15680000.0  15130000.0  16230000.0   
            2019                 SEDAN  15930000.0  15360000.0  16500000.0   
            2019                 SEDAN  16220000.0  15660000.0  16780000.0   
            2019             HATCHBACK  16520000.0  15940000.0  17100000.0   
            2019             HATCHBACK  16770000.0  15990000.0  17540000.0   
            2019             HATCHBACK  16820000.0  16040000.0  17590000.0   
            2019             HATCHBACK  17050000.0  16290000.0  17810000.0   
            2019             HATCHBACK  17180000.0  16400000.0  17950000.0   
            2019             HATCHBACK  17610000.0  16820000.0  18410000.0   

                        km_referencia  publicaciones  rank1  rank2  \
mark  model year_model                                               
MAZDA 3     2019              42000.0           24.0    1.0    1.0   
            2019              42000.0           24.0    2.0    1.0   
            2019              43000.0           27.0    3.0    2.0   
            2019              43000.0           27.0    4.0    2.0   
            2019              38000.0           35.0   

##### 3. Prepare and clean market data

    * Clean and parse transmission_type

In [11]:
#correct the name of the column
try:
    data_df.rename(columns={"transmition_type":"transmission_type"},inplace=True)
except:

    pass
#used for parsing
auto=["auto","automática","automático","automatic","automatico","semiautomática","a","at","automática secuencial","tiptronic"]
manual=["manual","mecánica","mecanico","mecánico","m","mt"]

def parse_data_transmission(row):
    transmission=row.transmission_type
    if not pd.isnull(transmission):
        transmission=transmission.lower()
        if transmission in auto:
            return "auto"
        if transmission in manual:
            return 'manual'
        # else :
        #     print("transmission -->",transmission)
        #     return transmission
            
    title=row.title
    if not pd.isnull(title):
        title=title.lower()
        for t in ["automática","automático","automatic","automatico","semiautomática","automática secuencial","tiptronic"]:
            if t in title:
                # print("auto from title ==>",title)
                return "auto"
        for t in ["manual","mecánica","mecanico","mecánico"]:
            if t in title:
                # print("manual from title ==>",title)
                return "manual"

    return np.nan

data_df['transmission_type']=data_df.apply(parse_data_transmission, axis='columns')


### 2- Match and complete market data  

In [12]:
#those are some global variablas that will controle the matching accuracy
mark_match_minscore=85
model_match_minscore=85
version_comp_match_minscore=68


    * construct a search index from models data easier search and access

In [13]:
#create a search index from models data to be used in matching between sheet1 and sheet2
search_index=models_df.reset_index().set_index(["mark","model","year_model",'transmission_type',"version_comp","price"])
search_index=search_index.index.to_frame()
search_index.sort_index(inplace=True)
search_index.drop_duplicates(inplace=True)

In [14]:
search_index.head(6)

mark    model  \
mark  model   year_model transmission_type version_comp price                   
ACURA INTEGRA 1990       auto              1.8 GS AT 4P NaN    ACURA  INTEGRA   
                         manual            1.6 LS MT 4P NaN    ACURA  INTEGRA   
                                           1.6 MT 4P    NaN    ACURA  INTEGRA   
                                           1.8 LS MT 4P NaN    ACURA  INTEGRA   
                                           1.8 MT 4P    NaN    ACURA  INTEGRA   
                                           1.8 RS MT 4P NaN    ACURA  INTEGRA   

                                                               year_model  \
mark  model   year_model transmission_type version_comp price               
ACURA INTEGRA 1990       auto              1.8 GS AT 4P NaN          1990   
                         manual            1.6 LS MT 4P NaN          1990   
                                           1.6 MT 4P    NaN          1990   
                                           1.8 LS MT 4P NaN          1990   
                                           1.8 MT 4P    NaN          1990   
                                           1.8 RS MT 4P NaN          1990   

                                                              transmission_type  \
mark  model   year_model transmission_type version_comp price                     
ACURA INTEGRA 1990       auto              1.8 GS AT 4P NaN                auto   
                         manual            1.6 LS MT 4P NaN              manual   
                                           1.6 MT 4P    NaN              manual   
                                           1.8 LS MT 4P NaN              manual   
                                           1.8 MT 4P    NaN              manual   
                                           1.8 RS MT 4P NaN              manual   

                                                               version_comp  \
mark  model   year_model transmission_type version_comp price                 
ACURA INTEGRA 1990       auto              1.8 GS AT 4P NaN    1.8 GS AT 4P   
                         manual            1.6 LS MT 4P NaN    1.6 LS MT 4P   
                                           1.6 MT 4P    NaN       1.6 MT 4P   
                                           1.8 LS MT 4P NaN    1.8 LS MT 4P   
                                           1.8 MT 4P    NaN       1.8 MT 4P   
                                           1.8 RS MT 4P NaN    1.8 RS MT 4P   

                                                               price  
mark  model   year_model transmission_type version_comp price         
ACURA INTEGRA 1990       auto              1.8 GS AT 4P NaN      NaN  
                         manual            1.6 LS MT 4P NaN      NaN  
                                           1.6 MT 4P    NaN      NaN  
                                           1.8 LS MT 4P NaN      NaN  
                                           1.8 MT 4P    NaN      NaN  
                                           1.8 RS MT 4P NaN      NaN

##### 1- Match and complete the "mark2" field 

    * Match the mark:

In [15]:
#get all available marks 
marks=search_index.index.get_level_values("mark").unique()
marks

Index(['ACURA', 'ALFA ROMEO', 'ASIA MOTORS', 'ASTON MARTIN', 'AUDI', 'AUSTIN',
       'AUTORRAD', 'BAIC', 'BENTLEY', 'BMW',
       ...
       'SUZUKI', 'TATA', 'TOYOTA', 'UAZ', 'VOLKSWAGEN', 'VOLVO', 'YUGO', 'ZNA',
       'ZOTYE', 'ZX AUTO'],
      dtype='object', name='mark', length=110)

In [16]:

#this will will use to do some dynamic programing to help us improve the speed of matching marks and models
mark_memo={}
#define the mark matching function
def match_mark(row):
    minscore=mark_match_minscore
    mark=row.mark
    choice=np.nan
    score=np.nan
    match=np.nan
    if not pd.isnull(mark):
        mark=mark
        match=mark_memo.get(mark)
        if not match:
            match = fwp.extractOne(mark,marks,score_cutoff=minscore)
            if match:
                mark_memo[mark]=match
    #
    if pd.isnull(match):
        #if mark field not found din the dataset try to findd a mark match in the tittle
        title=row.title
        if pd.isnull(title):
            return  np.nan,np.nan
        
        title = title
        p=re.compile("[a-zA-z]+")
        res=filter(p.match,title.split(' '))
        matches=[fwp.extractOne(cand,marks,score_cutoff=minscore) for cand in res if len(cand) >2 or cand.lower() in ['ds','zx','mg','sg']]
        #this will filter non values
        matches=[m for m in matches if m ]

        if matches:
            maxscore=0
            #find the match with the highest score
            for m in matches:
                if m[1] >maxscore:
                    match=m
                    maxscore=m[1]
        

    if  not pd.isnull(match):
        choice,score=match

        #for some special cases where the mark lengh is small (<=2,  like DS ),
        # ensure the match scoreis very hight to not get fals positives 
        if not pd.isnull(choice) and len(choice)<=2 and score<95:
            score=np.nan
            choice=np.nan
   
    return  choice,score 


In [17]:
#match mark 
mark_match_df = data_df.apply(match_mark, axis='columns', result_type='expand')
data_df['mark2']=mark_match_df[0]
# uncomment line bellow ifd you want the score
# data_df["mark2_match_score"]=mark_match_df[1]    

In [18]:
data_df.head(4)

,web,id,title,price,mark,type,model,model_year,transmission_type,fuel_type,version_comp,mark2,model2,model_year2,rank1,rank2
0,cha,GI-AD-22186,PEUGEOT 3008 2018 3008 ACTIVE 1.6 AUT,19790000,NaN,Station Wagon,3008,NaN,NaN,Bencina,NaN,PEUGEOT,NaN,NaN,NaN,NaN
1,cha,CL-AD-12305883,2021 Peugeot 208 1.2 Puretech Auto 130HP Premi...,19700000,PEUGEOT,hatchback,208,2021.0,auto,Bencina,NaN,PEUGEOT,NaN,NaN,NaN,NaN
2,cha,GI-AD-21746,TOYOTA HILUX 2021 HI LUX TM 4X4 2.8,36290000,NaN,Camioneta,hilux,NaN,NaN,Diesel,NaN,TOYOTA,NaN,NaN,NaN,NaN
3,cha,CL-AD-12285271,2012 Toyota Rav4,11500000,NaN,stationwagon,rav4,NaN,auto,Bencina,NaN,TOYOTA,NaN,NaN,NaN,NaN


##### 2- Match and complete the "model2" field 

    * Match the model:

In [19]:
models_memo={}
#define the model matching function

def match_model(row):
    minscore=model_match_minscore
    mark=row.mark2
    match=np.nan
    choice=np.nan
    score=np.nan
    #if no mark matched from pervious step skip model matching also
    if  pd.isnull(mark):
        return choice,score
    
    #get all mark availbale models
    mark_models=search_index.loc[mark].index.get_level_values("model").unique()

    #get the model value to match from  row
    model=row.model
   
    if  not  pd.isnull(model): 
        #get the model from cache if available 
        model=model
        match=models_memo.get(mark+"_"+model)
        if not match:  
            match=fwp.extractOne(model ,mark_models,score_cutoff=minscore)
            if match:
                #cache the result
                models_memo[mark+"_"+model]=match

    else:
        title=row.title
        if pd.isnull(title):
            return np.nan,np.nan
        title=title
        #calculate each model match score
        matches= fwp.extract(title ,mark_models)
        #filtes matches and keep only those with a score greater or equal to minscore
        matches=list(filter(lambda x: x[1]>=minscore,matches))
        if len(matches):
            match =matches[0]
            #if the len choise from the current match  is less than 2 then and the score is less than 100
            #try to find a better choise 
            if len(match[0])<=2 and match[1] <100:
                alternatives=list(filter(lambda x: len(x[0])>2,matches))
                if alternatives:
                    match = alternatives[0]

    if  not pd.isnull(match):
        choice,score=match

    return  choice,score

In [20]:
# match the model
model_match_df = data_df.apply(match_model, axis='columns', result_type='expand')
data_df['model2']=model_match_df[0]
# uncomment line bellow if you want the score
#data_df["model_match_score"]=model_match_df[1]    

In [21]:
data_df.head(3)

,web,id,title,price,mark,type,model,model_year,transmission_type,fuel_type,version_comp,mark2,model2,model_year2,rank1,rank2
0,cha,GI-AD-22186,PEUGEOT 3008 2018 3008 ACTIVE 1.6 AUT,19790000,NaN,Station Wagon,3008,NaN,NaN,Bencina,NaN,PEUGEOT,3008,NaN,NaN,NaN
1,cha,CL-AD-12305883,2021 Peugeot 208 1.2 Puretech Auto 130HP Premi...,19700000,PEUGEOT,hatchback,208,2021.0,auto,Bencina,NaN,PEUGEOT,208,NaN,NaN,NaN
2,cha,GI-AD-21746,TOYOTA HILUX 2021 HI LUX TM 4X4 2.8,36290000,NaN,Camioneta,hilux,NaN,NaN,Diesel,NaN,TOYOTA,HILUX,NaN,NaN,NaN


##### 3- Match and complete the "year2" field 

In [22]:
#define the year matching function
def match_model_year(row):
    # for the year the match needs to be an exact match
    mark=row.mark2
    model=row.model2
    year=row.model_year
    #only match if mark andd model zere matchedd from pervious steps
    if not pd.isnull(mark) and not pd.isnull(model) :    
        #get all  the years from that specific model 
        model_years=search_index.loc[mark,model]['year_model'].unique().astype(str)
        
        # # model_years=[str(y) for y in model_years]
        if  not pd.isnull(year) and str(year) in model_years:
            return int(year)
        # if the year value isn't set for the row, try with title  
        else :
            title=row.title
            # extract all 4 digit values from title
            years =re.findall("(\d{4})", title) 
            #return the first value that matches a model year
            for year in years :
                if year in model_years:
                    
                    return int(year)

    
    return  np.nan

In [23]:
## convert to string
##data_df['model_year2']=data_df['model_year2'].apply(str)

In [24]:
#match year2
data_df['model_year2'] = data_df.apply(match_model_year, axis='columns')

TypeError: expected string or bytes-like object

In [ ]:
data_df.head(5)

##### 4- Match and complete the "version_comp" field 

     4.1- PAHSE 1: Fuzzy String (from title)  Match :

In [ ]:
search_index.sort_index(inplace=True)
search_index.loc["ACURA","INTEGRA",1990,"manual"]

In [ ]:

def match_version_comp(row):
    try :
        minscore=version_comp_match_minscore
        mark=row.mark2
        model=row.model2
        year=row.model_year2
        transmission=row.transmission_type
        if pd.isnull(transmission):
            transmission=slice(None)
        # year=int(year)

        versions=search_index.loc[mark,model,year,transmission]['version_comp'].unique()
        # if there is only one version available simply return that version
        if len(versions)==1:
            return versions

        #get title 
        title=row.title
        matches = fwp.extract(title ,versions,scorer=fuzz.token_set_ratio,limit=20) 
        
        maxscore=matches[0][1]
        #if the max score foundd is less than the minscore than return nan
        if maxscore<minscore:
            return np.nan

        #filter matches andd keep only those with a valid score
        matches= [match[0] for match in matches if match[1]==maxscore]
         
        if matches:
            #if more than one match make sure the pricing diffrence (beween, the max and the min) is less than 5% 
            if len (matches)>1:
                df=search_index.loc[mark,model,year,transmission,matches].dropna()
                #if we are here that means we have more than 1 option

                mx=df["price"].max()
                mn=df["price"].min()
                change =(mx-mn)/mx*100

                #if the price diffrence between the options is larger than 5% skip matching
                if change>5:
                    return np.nan
                
                #if the change is less than 5%
                #GET ALL VERSION THAN HAS THE PRICING
                #this will make sure the version that has no pricing are droped
                matches=df['version_comp'].unique()
                # if len(matches)>1:
                #     print(matches)
            return  matches
        
        return np.nan
        
    except Exception:

        return np.nan
#back['version_comp']= back.apply(match_version_comp, axis='columns')



In [ ]:
data_df['version_comp']= data_df.apply(match_version_comp, axis='columns')
#explode the version that have more than one match to diffrens rows each with a diffrent version

data_df=data_df.explode('version_comp',ignore_index=True)

     4.2- PAHSE 2: Price matching (from PHASE 1) Match:

In [ ]:
#construct a sub data frame (from data) that only contains the rows with
#versions identified in phase 1 
# construct a data frame from matched versions in phase 1 to be usedd in phase 2 
ver_df=data_df[["mark2",'model2',"model_year2","price","transmission_type",'version_comp']]
ver_df=ver_df.dropna()
ver_df.set_index(["mark2",'model2',"model_year2","transmission_type"],inplace=True)
ver_df.sort_index(inplace=True)
ver_df.drop_duplicates(inplace=True)
ver_df.reset_index(inplace=True)
ver_df=ver_df.groupby(["mark2",'model2',"model_year2","transmission_type",'version_comp']).mean()
ver_df.sort_index(inplace=True)
ver_df

In [ ]:
ver_df.loc["BMW","Z4",2003]

In [ ]:
# define the matching function
def match_version_comp2(row):
    try :
        
        ver=row.version_comp
    
        if not pd.isnull(ver):
            # already matched in phase one, return the same version
            return ver,"title match"

        
        mark=row.mark2
        model=row.model2
        year=row.model_year2
        price=row.price
        transmission=row.transmission_type
        if pd.isnull(transmission):
            transmission=slice(None)
        #get all similar versions identified in phase 1
        similar_versions=ver_df.loc[mark,model,year,transmission]
        
        
        #find the distance in  price between current model and 
        #the mean price of each posible version 
        similar_versions["distance"]=similar_versions.apply(lambda x : abs(price-x.price),axis='columns')
        
        #find the minimum 
        min_distance=similar_versions["distance"].min()
        

        #change percentage 
        change= (min_distance/price)*100
        #if the change is lesss than 5%,skip

        if change>5:
            return np.nan, "min price distance change {:.2f} %".format(change)
        #find those version which have the min distance
        versions=similar_versions.index[similar_versions["distance"] ==min_distance].get_level_values("version_comp").unique()
    
         
        # print(np.unique(versions))
        return versions,"price match ,distance change {:.2f} %".format(change)
     

    except Exception as e:
        # raise e
        # if any exception means that there were no similar moddels available
        return np.nan,np.nan


In [ ]:
#match 
version_match2=data_df.apply(match_version_comp2, axis='columns', result_type='expand')
data_df['version_comp']=version_match2[0]
data_df['version match debug']=version_match2[1]


In [ ]:
data_df=data_df.explode('version_comp',ignore_index=True)

### JOIN AND SAVE THE FINAL RESULT


In [ ]:
models_df.reset_index(inplace=True)
models=models_df.copy()
models_df.drop(['modelo_id','tipo_carroc',"model_comp"],axis="columns",inplace=True)
models_df.rename(columns={"mark":"mark2","model":"model2","year_model":"model_year2",'price':'price2', "transmission_type":"transmission_type2"},inplace=True)
data_df.drop(["rank1","rank2"],axis="columns",inplace=True)
final_data=pd.merge(data_df,models_df,left_on=["mark2","model2","model_year2","version_comp"], right_on=["mark2","model2","model_year2","version_comp"],how="left")


In [ ]:
#save

writer = pd.ExcelWriter('data_output.xlsx', engine='xlsxwriter')
models.to_excel(writer, sheet_name='Sheet2_models')
final_data.to_excel(writer, sheet_name='Sheet1_data')
writer.save()